In [171]:
import pandas as pd
from kmodes.kprototypes import KPrototypes
from kmodes.kmodes import KModes
import numpy
import math 

#dados = pd.read_csv('duracao_em_funcao_de_todas_as_variaveis_v2_com_filtro_quartis_sem_tipo_processo.csv', sep=';')
dados = pd.read_csv('duracao_em_funcao_de_todas_as_variaveis_v2_com_filtro_quartis.csv', sep=';')

In [172]:
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

In [173]:
#pip install pm4py

In [174]:
import pm4py
from pm4py.objects.log.util import dataframe_utils
from pm4py.objects.conversion.log import converter as log_converter
from pm4py.algo.discovery.dfg import algorithm as dfg_discovery
from pm4py.visualization.dfg import visualizer as dfg_visualization
from pm4py.objects.conversion.dfg import converter as dfg_mining
from pm4py.visualization.petrinet import visualizer as pn_visualizer
from pm4py.algo.filtering.log.variants import variants_filter
from pm4py.statistics.traces.log import case_statistics
from pm4py.statistics.traces.log import case_arrival

In [175]:
dados.head(10)

,codtribunal,grau,cod_orgao_julg,tipo_justica,descricao,uf,Porte_Municipio,PIB_percapita,Classificacao_PIB_per_capita,tipo_processo,Porte_Tribunal,Atuacao_Vara,Classificacao_Acervo_2,Median(numero_dias),Count*(npu),Sum(numero_dias)
0,TJAC,G1,11674,Estadual,VARA CÍVEL,AC,Pequeno_Porte,9634.339844,Municipio com médio PIB per capita,Civel,PEQUENO,Civel Pura,Acervo Grande,626,52,38983
1,TJAC,G1,11674,Estadual,VARA CÍVEL,AC,Pequeno_Porte,9634.339844,Municipio com médio PIB per capita,Execução Fiscal,PEQUENO,Civel Pura,Acervo Grande,935,21,23059
2,TJAC,G1,11674,Estadual,VARA CÍVEL,AC,Pequeno_Porte,9634.339844,Municipio com médio PIB per capita,Família,PEQUENO,Civel Pura,Acervo Grande,209,12,4112
3,TJAC,G1,14075,Estadual,VARA DE EXECUÇÕES PENAS E MEDIDAS ALTERNATIVAS,AC,Grande_Porte,12542.309570,Municipio com médio PIB per capita,Criminal,PEQUENO,Criminal Pura,Acervo Grande,1041,79,95393
4,TJAC,G1,16317,Estadual,VARA ÚNICA,AC,Pequeno_Porte,8538.910156,Municipio com médio PIB per capita,Civel,PEQUENO,Geral,Acervo Grande,1869,2,4284
5,TJAC,G1,16317,Estadual,VARA ÚNICA,AC,Pequeno_Porte,8538.910156,Municipio com médio PIB per capita,Criminal,PEQUENO,Geral,Acervo Grande,1266,2,3828
6,TJAC,G1,17049,Estadual,VARA ÚNICA,AC,Pequeno_Porte,9114.250000,Municipio com médio PIB per capita,Civel,PEQUENO,Geral,Acervo Grande,498,24,15104
7,TJAC,G1,17049,Estadual,VARA ÚNICA,AC,Pequeno_Porte,9114.250000,Municipio com médio PIB per capita,Criminal,PEQUENO,Geral,Acervo Grande,777,31,24777
8,TJAC,G1,17049,Estadual,VARA ÚNICA,AC,Pequeno_Porte,9114.250000,Municipio com médio PIB per capita,Execução Fiscal,PEQUENO,Geral,Acervo Grande,511,26,22410
9,TJAC,G1,17049,Estadual,VARA ÚNICA,AC,Pequeno_Porte,9114.250000,Municipio com médio PIB per capita,Família,PEQUENO,Geral,Acervo Grande,217,14,5317


In [176]:
list(dados.columns)

['codtribunal',
 'grau',
 'cod_orgao_julg',
 'tipo_justica',
 'descricao',
 'uf',
 'Porte_Municipio',
 'PIB_percapita',
 'Classificacao_PIB_per_capita',
 'tipo_processo',
 'Porte_Tribunal',
 'Atuacao_Vara',
 'Classificacao_Acervo_2',
 'Median(numero_dias)',
 'Count*(npu)',
 'Sum(numero_dias)']

In [177]:
#colunas = ["Atuacao_Vara","Median(numero_dias)","Count*(npu)","PIB_percapita","Pop_est_2009"]
colunas = ["Porte_Municipio","Porte_Tribunal","Classificacao_Acervo_2","Atuacao_Vara","Classificacao_PIB_per_capita"]
colunas_str = "Porte_Municipio;Porte_Tribunal;Classificacao_Acervo_2;Atuacao_Vara;Classificacao_PIB_per_capita"
#colunas_num = ["Median(numero_dias)","Count*(npu)","PIB_percapita","Pop_est_2009"]
colunas_num = []
#colunas_cat_nomes = ["tipo_justica","Porte_Municipio","Atuacao_Vara","Classificacao_Acervo_2",]
colunas_cat = [0, 1, 2, 3, 4 ]

#renomear civel pura e criminal pura
dados['Atuacao_Vara'] = dados['Atuacao_Vara'].replace(['Civel Pura'],'Civel')
dados['Atuacao_Vara'] = dados['Atuacao_Vara'].replace(['Criminal Pura'],'Criminal')

#Preenche o porte do tribunal como nao_classifcado
dados['Porte_Tribunal'] = dados['Porte_Tribunal'].fillna('Nao_classificado')


print(dados.head(100))
resultado = pd.DataFrame(columns=['Cluster_config','resultado'])

   codtribunal grau  cod_orgao_julg tipo_justica  \
0         TJAC   G1           11674     Estadual   
1         TJAC   G1           11674     Estadual   
2         TJAC   G1           11674     Estadual   
3         TJAC   G1           14075     Estadual   
4         TJAC   G1           16317     Estadual   
..         ...  ...             ...          ...   
95        TJAC   G2           33563     Estadual   
96        TJAC   G2           74296     Estadual   
97        TJAC   JE           11674     Estadual   
98        TJAC   JE           11674     Estadual   
99        TJAC   JE           16426     Estadual   

                                         descricao  uf Porte_Municipio  \
0                                       VARA CÍVEL  AC   Pequeno_Porte   
1                                       VARA CÍVEL  AC   Pequeno_Porte   
2                                       VARA CÍVEL  AC   Pequeno_Porte   
3   VARA DE EXECUÇÕES PENAS E MEDIDAS ALTERNATIVAS  AC    Grande_Porte   
4    

In [178]:
nome_justica = 'Estadual'
atuacao = 'Civel'
dados_segmento = dados
dados_segmento = dados_segmento[dados_segmento['tipo_justica'] == nome_justica]
#dados = dados[dados['codtribunal'] == 'TJPE']
dados_segmento = dados_segmento[dados_segmento['Atuacao_Vara'] == atuacao]
print(dados_segmento.shape)
dados_filter = dados_segmento.filter(items=colunas)
#dados_filter['Median(numero_dias)'] = pd.to_numeric(dados_filter['Median(numero_dias)'])
#dados_filter['PIB_percapita'] = pd.to_numeric(dados_filter['PIB_percapita'])
#dados_filter['Count*(npu)'] = pd.to_numeric(dados_filter['Count*(npu)'])
print(dados_filter.dtypes)
print(dados_filter.head(100))

(1770, 16)
Porte_Municipio                 object
Porte_Tribunal                  object
Classificacao_Acervo_2          object
Atuacao_Vara                    object
Classificacao_PIB_per_capita    object
dtype: object
    Porte_Municipio Porte_Tribunal Classificacao_Acervo_2 Atuacao_Vara  \
0     Pequeno_Porte        PEQUENO          Acervo Grande        Civel   
1     Pequeno_Porte        PEQUENO          Acervo Grande        Civel   
2     Pequeno_Porte        PEQUENO          Acervo Grande        Civel   
14     Grande_Porte        PEQUENO          Acervo Grande        Civel   
25    Pequeno_Porte        PEQUENO          Acervo Grande        Civel   
..              ...            ...                    ...          ...   
277    Grande_Porte        PEQUENO          Acervo Grande        Civel   
278    Grande_Porte        PEQUENO          Acervo Grande        Civel   
279    Grande_Porte        PEQUENO          Acervo Grande        Civel   
319    Grande_Porte        PEQUENO      

In [179]:
for x in range(3,9):
    qtd_clusters = x
    #km = KPrototypes(n_clusters=qtd_clusters, init='Huang', n_init=5, verbose=1)
    km = KModes(n_clusters=qtd_clusters, init='Huang', n_init=5, verbose=1)
    clusters = km.fit_predict(dados_filter, categorical=colunas_cat)
    print('Centroídes')
    print(km.cluster_centroids_)
    print('Clusters')
    print(clusters)
    #dados_cluster = dados_filter
    #dados_cluster['Cluster'] = clusters
    Nome_cluster = 'jus_' + nome_justica + '_atuacao_' + atuacao + '_cluster_' + str(qtd_clusters)
    dados_segmento[Nome_cluster] = clusters
    print(dados_segmento.head(10))
    numpy.savetxt('Clusters_centroides_' + Nome_cluster + '.csv', km.cluster_centroids_ , fmt = '%s', delimiter=";", header=colunas_str, comments='')
    #Agrupamento
    print("Agrupamento...")
    print(str(x))
    dados_agrupados = dados_segmento.filter(items=[Nome_cluster,'Count*(npu)','Sum(numero_dias)']).groupby(by=[Nome_cluster]).sum()
    
    dados_agrupados['media_duracao'] = dados_agrupados['Sum(numero_dias)'] / dados_agrupados['Count*(npu)']
    print(dados_agrupados)
    atual = 0
    anterior = 0
    valor = 0
    media_quad_dif = 0
    for index, row in dados_agrupados.iterrows():
        anterior = atual
        atual = row['media_duracao']
        if anterior > 0:
            valor += pow(anterior-atual, 2)
    media_quad_dif = math.sqrt(valor/qtd_clusters)
    temp = pd.DataFrame({'Cluster_config':[Nome_cluster],'resultado':[media_quad_dif]})
    resultado = resultado.append(temp)
    
#sns.lmplot(data=dados_cluster.filter(items=colunas_num), x='Median(numero_dias)', y='Count*(npu)', hue='Cluster', 
#                   fit_reg=False, legend=True, legend_out=True)

dados_segmento.to_csv('Clusters_jus_' + nome_justica + '_atuacao_' + atuacao +'_classificacoes.csv', sep = ";")
col = ['codtribunal','cod_orgao_julg','descricao','Cluster_3','Cluster_4','Cluster_5','Cluster_6','Cluster_7','Cluster_8']
dados_somente_orgao_julgador_e_cluster = dados_segmento.filter(items=col)
dados_somente_orgao_julgador_e_cluster.to_csv('Clusters_jus_' + nome_justica + '_atuacao_' + atuacao +'_classificacoes_clusters_somente_orgao_julgador_e cluster.csv', sep = ";")

Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 1, iteration: 1/100, moves: 702, cost: 1577.0
Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 2, iteration: 1/100, moves: 138, cost: 1505.0
Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 3, iteration: 1/100, moves: 57, cost: 1565.0
Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 4, iteration: 1/100, moves: 191, cost: 1622.0
Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 5, iteration: 1/100, moves: 228, cost: 1624.0
Best run was number 2
Centroídes
[['Grande_Porte' 'MÉDIO' 'Acervo Grande' 'Civel'
  'Municipio com alto PIB per capita']
 ['Grande_Porte' 'MÉDIO' 'Acervo Grande' 'Civel'
  'Municipio com médio PIB per capita']
 ['Grande_Porte' 'PEQUENO' 'Acervo Grande' 'Civel'
  'Municipio com médio PIB per capita']]
Clusters
[2 2 2 ... 0 2 2]
   codtribunal grau  c

Starting iterations...
Run 1, iteration: 1/100, moves: 362, cost: 1214.0
Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 2, iteration: 1/100, moves: 199, cost: 1246.0
Run 2, iteration: 2/100, moves: 42, cost: 1246.0
Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 3, iteration: 1/100, moves: 376, cost: 1025.0
Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 4, iteration: 1/100, moves: 451, cost: 1167.0
Run 4, iteration: 2/100, moves: 10, cost: 1167.0
Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 5, iteration: 1/100, moves: 227, cost: 1128.0
Run 5, iteration: 2/100, moves: 65, cost: 1128.0
Best run was number 3
Centroídes
[['Grande_Porte' 'MÉDIO' 'Acervo Grande' 'Civel'
  'Municipio com alto PIB per capita']
 ['Medio_Porte' 'PEQUENO' 'Acervo Grande' 'Civel'
  'Municipio com médio PIB per capita']
 ['Grande_Porte' 'GRANDE' 'Acervo Médio' 'Civel'


Run 1, iteration: 1/100, moves: 525, cost: 923.0
Run 1, iteration: 2/100, moves: 117, cost: 923.0
Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 2, iteration: 1/100, moves: 303, cost: 945.0
Run 2, iteration: 2/100, moves: 55, cost: 945.0
Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 3, iteration: 1/100, moves: 422, cost: 792.0
Run 3, iteration: 2/100, moves: 41, cost: 792.0
Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 4, iteration: 1/100, moves: 125, cost: 906.0
Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 5, iteration: 1/100, moves: 427, cost: 892.0
Best run was number 3
Centroídes
[['Pequeno_Porte' 'PEQUENO' 'Acervo Grande' 'Civel'
  'Municipio com baixo PIB per capita']
 ['Grande_Porte' 'MÉDIO' 'Acervo Grande' 'Civel'
  'Municipio com médio PIB per capita']
 ['Medio_Porte' 'PEQUENO' 'Acervo Grande' 'Civel'
  'Municipio com médio PIB

Run 3, iteration: 1/100, moves: 97, cost: 816.0
Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 4, iteration: 1/100, moves: 260, cost: 953.0
Run 4, iteration: 2/100, moves: 29, cost: 953.0
Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 5, iteration: 1/100, moves: 122, cost: 1042.0
Best run was number 3
Centroídes
[['Medio_Porte' 'PEQUENO' 'Acervo Grande' 'Civel'
  'Municipio com médio PIB per capita']
 ['Grande_Porte' 'MÉDIO' 'Acervo Grande' 'Civel'
  'Municipio com alto PIB per capita']
 ['Grande_Porte' 'MÉDIO' 'Acervo Médio' 'Civel'
  'Municipio com alto PIB per capita']
 ['Pequeno_Porte' 'PEQUENO' 'Acervo Grande' 'Civel'
  'Municipio com médio PIB per capita']
 ['Medio_Porte' 'MÉDIO' 'Acervo Grande' 'Civel'
  'Municipio com alto PIB per capita']
 ['Grande_Porte' 'PEQUENO' 'Acervo Grande' 'Civel'
  'Municipio com médio PIB per capita']
 ['Grande_Porte' 'GRANDE' 'Acervo Grande' 'Civel'
  'Municipio com alto PIB p

In [180]:
nome_justica = 'Estadual'
atuacao = 'Criminal'
dados_segmento = dados
dados_segmento = dados_segmento[dados_segmento['tipo_justica'] == nome_justica]
#dados = dados[dados['codtribunal'] == 'TJPE']
dados_segmento = dados_segmento[dados_segmento['Atuacao_Vara'] == atuacao]
print(dados_segmento.shape)
dados_filter = dados_segmento.filter(items=colunas)
#dados_filter['Median(numero_dias)'] = pd.to_numeric(dados_filter['Median(numero_dias)'])
#dados_filter['PIB_percapita'] = pd.to_numeric(dados_filter['PIB_percapita'])
#dados_filter['Count*(npu)'] = pd.to_numeric(dados_filter['Count*(npu)'])
print(dados_filter.dtypes)

(615, 16)
Porte_Municipio                 object
Porte_Tribunal                  object
Classificacao_Acervo_2          object
Atuacao_Vara                    object
Classificacao_PIB_per_capita    object
dtype: object


In [181]:
for x in range(3,9):
    qtd_clusters = x
    #km = KPrototypes(n_clusters=qtd_clusters, init='Huang', n_init=5, verbose=1)
    km = KModes(n_clusters=qtd_clusters, init='Huang', n_init=5, verbose=1)
    clusters = km.fit_predict(dados_filter, categorical=colunas_cat)
    print('Centroídes')
    print(km.cluster_centroids_)
    print('Clusters')
    print(clusters)
    dados_cluster = dados_filter
    #dados_cluster['Cluster'] = clusters
    Nome_cluster = 'jus_' + nome_justica + '_atuacao_' + atuacao + '_cluster_' + str(qtd_clusters)
    dados_segmento[Nome_cluster] = clusters
    print(dados_segmento.head(10))
    numpy.savetxt('Clusters_centroides_' + Nome_cluster + '.csv', km.cluster_centroids_ , fmt = '%s', delimiter=";", header=colunas_str, comments='')
    #Agrupamento
    print("Agrupamento...")
    print(str(x))
    dados_agrupados = dados_segmento.filter(items=[Nome_cluster,'Count*(npu)','Sum(numero_dias)']).groupby(by=[Nome_cluster]).sum()
    
    dados_agrupados['media_duracao'] = dados_agrupados['Sum(numero_dias)'] / dados_agrupados['Count*(npu)']
    print(dados_agrupados)
    atual = 0
    anterior = 0
    valor = 0
    media_quad_dif = 0
    for index, row in dados_agrupados.iterrows():
        anterior = atual
        atual = row['media_duracao']
        if anterior > 0:
            valor += pow(anterior-atual, 2)
    media_quad_dif = math.sqrt(valor/qtd_clusters)
    temp = pd.DataFrame({'Cluster_config':[Nome_cluster],'resultado':[media_quad_dif]})
    resultado = resultado.append(temp)
    
#sns.lmplot(data=dados_cluster.filter(items=colunas_num), x='Median(numero_dias)', y='Count*(npu)', hue='Cluster', 
#                   fit_reg=False, legend=True, legend_out=True)

dados_segmento.to_csv('Clusters_jus_' + nome_justica + '_atuacao_' + atuacao +'_classificacoes.csv', sep = ";")
col = ['codtribunal','cod_orgao_julg','descricao','Cluster_3','Cluster_4','Cluster_5','Cluster_6','Cluster_7','Cluster_8']
dados_somente_orgao_julgador_e_cluster = dados_segmento.filter(items=col)
dados_somente_orgao_julgador_e_cluster.to_csv('Clusters_jus_' + nome_justica + '_atuacao_' + atuacao +'_classificacoes_clusters_somente_orgao_julgador_e cluster.csv', sep = ";")

Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 1, iteration: 1/100, moves: 126, cost: 583.0
Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 2, iteration: 1/100, moves: 135, cost: 534.0
Run 2, iteration: 2/100, moves: 18, cost: 534.0
Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 3, iteration: 1/100, moves: 154, cost: 573.0
Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 4, iteration: 1/100, moves: 82, cost: 628.0
Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 5, iteration: 1/100, moves: 176, cost: 542.0
Best run was number 2
Centroídes
[['Grande_Porte' 'MÉDIO' 'Acervo Grande' 'Criminal'
  'Municipio com alto PIB per capita']
 ['Grande_Porte' 'PEQUENO' 'Acervo Grande' 'Criminal'
  'Municipio com médio PIB per capita']
 ['Medio_Porte' 'MÉDIO' 'Acervo Médio' 'Criminal'
  'Municipio com médio PIB per capita']]

Starting iterations...
Run 1, iteration: 1/100, moves: 88, cost: 418.0
Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 2, iteration: 1/100, moves: 70, cost: 461.0
Run 2, iteration: 2/100, moves: 5, cost: 461.0
Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 3, iteration: 1/100, moves: 164, cost: 520.0
Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 4, iteration: 1/100, moves: 70, cost: 500.0
Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 5, iteration: 1/100, moves: 136, cost: 387.0
Best run was number 5
Centroídes
[['Grande_Porte' 'MÉDIO' 'Acervo Médio' 'Criminal'
  'Municipio com médio PIB per capita']
 ['Grande_Porte' 'MÉDIO' 'Acervo Grande' 'Criminal'
  'Municipio com alto PIB per capita']
 ['Medio_Porte' 'PEQUENO' 'Acervo Grande' 'Criminal'
  'Municipio com médio PIB per capita']
 ['Grande_Porte' 'PEQUENO' 'Acervo Grande' 'Criminal'
  'Mun

Run 1, iteration: 1/100, moves: 0, cost: 432.0
Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 2, iteration: 1/100, moves: 107, cost: 353.0
Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 3, iteration: 1/100, moves: 83, cost: 418.0
Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 4, iteration: 1/100, moves: 42, cost: 447.0
Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 5, iteration: 1/100, moves: 52, cost: 424.0
Best run was number 2
Centroídes
[['Grande_Porte' 'MÉDIO' 'Acervo Grande' 'Criminal'
  'Municipio com alto PIB per capita']
 ['Medio_Porte' 'MÉDIO' 'Acervo Médio' 'Criminal'
  'Municipio com médio PIB per capita']
 ['Medio_Porte' 'PEQUENO' 'Acervo Grande' 'Criminal'
  'Municipio com médio PIB per capita']
 ['Grande_Porte' 'PEQUENO' 'Acervo Grande' 'Criminal'
  'Municipio com médio PIB per capita']
 ['Pequeno_Porte' 'MÉDIO' 'Acervo Médio

Run 1, iteration: 1/100, moves: 26, cost: 397.0
Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 2, iteration: 1/100, moves: 50, cost: 342.0
Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 3, iteration: 1/100, moves: 94, cost: 409.0
Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 4, iteration: 1/100, moves: 179, cost: 347.0
Run 4, iteration: 2/100, moves: 75, cost: 329.0
Run 4, iteration: 3/100, moves: 12, cost: 329.0
Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 5, iteration: 1/100, moves: 119, cost: 389.0
Run 5, iteration: 2/100, moves: 12, cost: 389.0
Best run was number 4
Centroídes
[['Medio_Porte' 'PEQUENO' 'Acervo Grande' 'Criminal'
  'Municipio com médio PIB per capita']
 ['Pequeno_Porte' 'MÉDIO' 'Acervo Grande' 'Criminal'
  'Municipio com alto PIB per capita']
 ['Grande_Porte' 'MÉDIO' 'Acervo Grande' 'Criminal'
  'Municipio com alto PI

Init: initializing clusters
Starting iterations...
Run 1, iteration: 1/100, moves: 95, cost: 292.0
Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 2, iteration: 1/100, moves: 0, cost: 309.0
Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 3, iteration: 1/100, moves: 80, cost: 358.0
Run 3, iteration: 2/100, moves: 5, cost: 358.0
Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 4, iteration: 1/100, moves: 146, cost: 343.0
Run 4, iteration: 2/100, moves: 5, cost: 343.0
Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 5, iteration: 1/100, moves: 38, cost: 364.0
Best run was number 1
Centroídes
[['Medio_Porte' 'MÉDIO' 'Acervo Grande' 'Criminal'
  'Municipio com alto PIB per capita']
 ['Pequeno_Porte' 'MÉDIO' 'Acervo Grande' 'Criminal'
  'Municipio com médio PIB per capita']
 ['Grande_Porte' 'PEQUENO' 'Acervo Grande' 'Criminal'
  'Municipio com médio PI

In [182]:
nome_justica = 'Estadual'
atuacao = 'Geral'
dados_segmento = dados
dados_segmento = dados_segmento[dados_segmento['tipo_justica'] == nome_justica]
#dados = dados[dados['codtribunal'] == 'TJPE']
dados_segmento = dados_segmento[dados_segmento['Atuacao_Vara'] == atuacao]
print(dados_segmento.shape)
dados_filter = dados_segmento.filter(items=colunas)
#dados_filter['Median(numero_dias)'] = pd.to_numeric(dados_filter['Median(numero_dias)'])
#dados_filter['PIB_percapita'] = pd.to_numeric(dados_filter['PIB_percapita'])
#dados_filter['Count*(npu)'] = pd.to_numeric(dados_filter['Count*(npu)'])
print(dados_filter.dtypes)

(2081, 16)
Porte_Municipio                 object
Porte_Tribunal                  object
Classificacao_Acervo_2          object
Atuacao_Vara                    object
Classificacao_PIB_per_capita    object
dtype: object


In [183]:
for x in range(3,9):
    qtd_clusters = x
    #km = KPrototypes(n_clusters=qtd_clusters, init='Huang', n_init=5, verbose=1)
    km = KModes(n_clusters=qtd_clusters, init='Huang', n_init=5, verbose=1)
    clusters = km.fit_predict(dados_filter, categorical=colunas_cat)
    print('Centroídes')
    print(km.cluster_centroids_)
    print('Clusters')
    print(clusters)
    dados_cluster = dados_filter
    #dados_cluster['Cluster'] = clusters
    Nome_cluster = 'jus_' + nome_justica + '_atuacao_' + atuacao + '_cluster_' + str(qtd_clusters)
    dados_segmento[Nome_cluster] = clusters
    print(dados_segmento.head(10))
    numpy.savetxt('Clusters_centroides_' + Nome_cluster + '.csv', km.cluster_centroids_ , fmt = '%s', delimiter=";", header=colunas_str, comments='')
    #Agrupamento
    print("Agrupamento...")
    print(str(x))
    dados_agrupados = dados_segmento.filter(items=[Nome_cluster,'Count*(npu)','Sum(numero_dias)']).groupby(by=[Nome_cluster]).sum()
    
    dados_agrupados['media_duracao'] = dados_agrupados['Sum(numero_dias)'] / dados_agrupados['Count*(npu)']
    print(dados_agrupados)
    atual = 0
    anterior = 0
    valor = 0
    media_quad_dif = 0
    for index, row in dados_agrupados.iterrows():
        anterior = atual
        atual = row['media_duracao']
        if anterior > 0:
            valor += pow(anterior-atual, 2)
    media_quad_dif = math.sqrt(valor/qtd_clusters)
    temp = pd.DataFrame({'Cluster_config':[Nome_cluster],'resultado':[media_quad_dif]})
    resultado = resultado.append(temp)
    
#sns.lmplot(data=dados_cluster.filter(items=colunas_num), x='Median(numero_dias)', y='Count*(npu)', hue='Cluster', 
#                   fit_reg=False, legend=True, legend_out=True)

dados_segmento.to_csv('Clusters_jus_' + nome_justica + '_atuacao_' + atuacao +'_classificacoes.csv', sep = ";")
col = ['codtribunal','cod_orgao_julg','descricao','Cluster_3','Cluster_4','Cluster_5','Cluster_6','Cluster_7','Cluster_8']
dados_somente_orgao_julgador_e_cluster = dados_segmento.filter(items=col)
dados_somente_orgao_julgador_e_cluster.to_csv('Clusters_jus_' + nome_justica + '_atuacao_' + atuacao +'_classificacoes_clusters_somente_orgao_julgador_e cluster.csv', sep = ";")

Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 1, iteration: 1/100, moves: 0, cost: 2039.0
Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 2, iteration: 1/100, moves: 467, cost: 1924.0
Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 3, iteration: 1/100, moves: 131, cost: 1856.0
Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 4, iteration: 1/100, moves: 485, cost: 2039.0
Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 5, iteration: 1/100, moves: 0, cost: 1790.0
Best run was number 5
Centroídes
[['Pequeno_Porte' 'PEQUENO' 'Acervo Grande' 'Geral'
  'Municipio com médio PIB per capita']
 ['Pequeno_Porte' 'MÉDIO' 'Acervo Médio' 'Geral'
  'Municipio com alto PIB per capita']
 ['Medio_Porte' 'MÉDIO' 'Acervo Grande' 'Geral'
  'Municipio com médio PIB per capita']]
Clusters
[0 0 0 ... 0 0 0]
   codtribunal grau  cod_

Run 1, iteration: 1/100, moves: 379, cost: 1396.0
Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 2, iteration: 1/100, moves: 0, cost: 1404.0
Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 3, iteration: 1/100, moves: 506, cost: 1397.0
Run 3, iteration: 2/100, moves: 68, cost: 1397.0
Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 4, iteration: 1/100, moves: 476, cost: 1602.0
Run 4, iteration: 2/100, moves: 195, cost: 1602.0
Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 5, iteration: 1/100, moves: 446, cost: 1403.0
Run 5, iteration: 2/100, moves: 100, cost: 1403.0
Best run was number 1
Centroídes
[['Pequeno_Porte' 'PEQUENO' 'Acervo Grande' 'Geral'
  'Municipio com médio PIB per capita']
 ['Medio_Porte' 'MÉDIO' 'Acervo Grande' 'Geral'
  'Municipio com baixo PIB per capita']
 ['Pequeno_Porte' 'PEQUENO' 'Acervo Grande' 'Geral'
  'Municipio com b

Init: initializing clusters
Starting iterations...
Run 1, iteration: 1/100, moves: 260, cost: 1301.0
Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 2, iteration: 1/100, moves: 285, cost: 1150.0
Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 3, iteration: 1/100, moves: 809, cost: 1257.0
Run 3, iteration: 2/100, moves: 470, cost: 1156.0
Run 3, iteration: 3/100, moves: 40, cost: 1156.0
Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 4, iteration: 1/100, moves: 190, cost: 1144.0
Run 4, iteration: 2/100, moves: 11, cost: 1144.0
Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 5, iteration: 1/100, moves: 0, cost: 1107.0
Best run was number 5
Centroídes
[['Medio_Porte' 'MÉDIO' 'Acervo Médio' 'Geral'
  'Municipio com baixo PIB per capita']
 ['Medio_Porte' 'MÉDIO' 'Acervo Grande' 'Geral'
  'Municipio com baixo PIB per capita']
 ['Pequeno_Porte' 'PEQUEN

Starting iterations...
Run 4, iteration: 1/100, moves: 137, cost: 1197.0
Run 4, iteration: 2/100, moves: 30, cost: 1197.0
Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 5, iteration: 1/100, moves: 210, cost: 1103.0
Best run was number 3
Centroídes
[['Medio_Porte' 'PEQUENO' 'Acervo Grande' 'Geral'
  'Municipio com médio PIB per capita']
 ['Pequeno_Porte' 'PEQUENO' 'Acervo Médio' 'Geral'
  'Municipio com médio PIB per capita']
 ['Medio_Porte' 'MÉDIO' 'Acervo Médio' 'Geral'
  'Municipio com médio PIB per capita']
 ['Pequeno_Porte' 'MÉDIO' 'Acervo Grande' 'Geral'
  'Municipio com alto PIB per capita']
 ['Medio_Porte' 'MÉDIO' 'Acervo Grande' 'Geral'
  'Municipio com baixo PIB per capita']
 ['Pequeno_Porte' 'PEQUENO' 'Acervo Grande' 'Geral'
  'Municipio com baixo PIB per capita']
 ['Medio_Porte' 'MÉDIO' 'Acervo Médio' 'Geral'
  'Municipio com baixo PIB per capita']
 ['Pequeno_Porte' 'PEQUENO' 'Acervo Grande' 'Geral'
  'Municipio com médio PIB per capita']

In [184]:
nome_justica = 'Federal'
atuacao = 'Civel'
dados_segmento = dados
dados_segmento = dados_segmento[dados_segmento['tipo_justica'] == nome_justica]
#dados = dados[dados['codtribunal'] == 'TJPE']
dados_segmento = dados_segmento[dados_segmento['Atuacao_Vara'] == atuacao]
print(dados_segmento.shape)
dados_filter = dados_segmento.filter(items=colunas)
#dados_filter['Median(numero_dias)'] = pd.to_numeric(dados_filter['Median(numero_dias)'])
#dados_filter['PIB_percapita'] = pd.to_numeric(dados_filter['PIB_percapita'])
#dados_filter['Count*(npu)'] = pd.to_numeric(dados_filter['Count*(npu)'])
print(dados_filter.dtypes)

(151, 16)
Porte_Municipio                 object
Porte_Tribunal                  object
Classificacao_Acervo_2          object
Atuacao_Vara                    object
Classificacao_PIB_per_capita    object
dtype: object


In [185]:
for x in range(3,9):
    qtd_clusters = x
    #km = KPrototypes(n_clusters=qtd_clusters, init='Huang', n_init=5, verbose=1)
    km = KModes(n_clusters=qtd_clusters, init='Huang', n_init=5, verbose=1)
    clusters = km.fit_predict(dados_filter, categorical=colunas_cat)
    print('Centroídes')
    print(km.cluster_centroids_)
    print('Clusters')
    print(clusters)
    dados_cluster = dados_filter
    #dados_cluster['Cluster'] = clusters
    Nome_cluster = 'jus_' + nome_justica + '_atuacao_' + atuacao + '_cluster_' + str(qtd_clusters)
    dados_segmento[Nome_cluster] = clusters
    print(dados_segmento.head(10))
    numpy.savetxt('Clusters_centroides_' + Nome_cluster + '.csv', km.cluster_centroids_ , fmt = '%s', delimiter=";", header=colunas_str, comments='')
    #Agrupamento
    print("Agrupamento...")
    print(str(x))
    dados_agrupados = dados_segmento.filter(items=[Nome_cluster,'Count*(npu)','Sum(numero_dias)']).groupby(by=[Nome_cluster]).sum()
    
    dados_agrupados['media_duracao'] = dados_agrupados['Sum(numero_dias)'] / dados_agrupados['Count*(npu)']
    print(dados_agrupados)
    atual = 0
    anterior = 0
    valor = 0
    media_quad_dif = 0
    for index, row in dados_agrupados.iterrows():
        anterior = atual
        atual = row['media_duracao']
        if anterior > 0:
            valor += pow(anterior-atual, 2)
    media_quad_dif = math.sqrt(valor/qtd_clusters)
    temp = pd.DataFrame({'Cluster_config':[Nome_cluster],'resultado':[media_quad_dif]})
    resultado = resultado.append(temp)
    
#sns.lmplot(data=dados_cluster.filter(items=colunas_num), x='Median(numero_dias)', y='Count*(npu)', hue='Cluster', 
#                   fit_reg=False, legend=True, legend_out=True)

dados_segmento.to_csv('Clusters_jus_' + nome_justica + '_atuacao_' + atuacao +'_classificacoes.csv', sep = ";")
col = ['codtribunal','cod_orgao_julg','descricao','Cluster_3','Cluster_4','Cluster_5','Cluster_6','Cluster_7','Cluster_8']
dados_somente_orgao_julgador_e_cluster = dados_segmento.filter(items=col)
dados_somente_orgao_julgador_e_cluster.to_csv('Clusters_jus_' + nome_justica + '_atuacao_' + atuacao +'_classificacoes_clusters_somente_orgao_julgador_e cluster.csv', sep = ";")

Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 1, iteration: 1/0, moves: 0, cost: 0.0
Centroídes
[['Grande_Porte' 'Nao_classificado' 'Acervo Grande' 'Civel'
  'Municipio com alto PIB per capita']
 ['Grande_Porte' 'Nao_classificado' 'Acervo Médio' 'Civel'
  'Municipio com alto PIB per capita']
 ['Grande_Porte' 'Nao_classificado' 'Acervo Médio' 'Civel'
  'Municipio com médio PIB per capita']]
Clusters
[1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 0 0 0 0 0 0 0 0 1 0 0 1 0 1 1 1 1 1 0
 1 1 0 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 0 0 0 1 1 1 0 0 0 0 0 0 1 1 1 1 0 0
 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 0 0 0 1 1 1 1 1 0 1 1 1 1 1 0 0 0 0 0
 2 2 2]
     codtribunal grau  cod_orgao_julg tipo_justica  \
5542        TRF2   G1           12241      Federal   
5543        TRF2   G1           12242      Federal   
5544        TRF2   G1           12242      Federal   
5545        TRF2   G1           1224

  'Municipio com médio PIB per capita']]
Clusters
[1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 0 0 0 0 0 0 0 0 1 0 0 1 0 1 1 1 1 1 0
 1 1 0 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 0 0 0 1 1 1 0 0 0 0 0 0 1 1 1 1 0 0
 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 0 0 0 1 1 1 1 1 0 1 1 1 1 1 0 0 0 0 0
 2 2 2]
     codtribunal grau  cod_orgao_julg tipo_justica  \
5542        TRF2   G1           12241      Federal   
5543        TRF2   G1           12242      Federal   
5544        TRF2   G1           12242      Federal   
5545        TRF2   G1           12245      Federal   
5546        TRF2   G1           12245      Federal   
5547        TRF2   G1           12246      Federal   
5548        TRF2   G1           12246      Federal   
5550        TRF2   G1           12251      Federal   
5551        TRF2   G1           12252      Federal   
5553        TRF2   G1           12255      Federal   

                             descricao  uf Port

[10 rows x 21 columns]
Agrupamento...
7
                                     Count*(npu)  Sum(numero_dias)  \
jus_Federal_atuacao_Civel_cluster_7                                  
0                                           2153           4043622   
1                                            435            707051   
2                                              5               702   

                                     media_duracao  
jus_Federal_atuacao_Civel_cluster_7                 
0                                      1878.133767  
1                                      1625.404598  
2                                       140.400000  
Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 1, iteration: 1/0, moves: 0, cost: 0.0
Centroídes
[['Grande_Porte' 'Nao_classificado' 'Acervo Grande' 'Civel'
  'Municipio com alto PIB per capita']
 ['Grande_Porte' 'Nao_classificado' 'Acervo Médio' 'Civel'
  'Municipio com alto PIB per capita']
 ['Grande_Por

In [186]:
nome_justica = 'Federal'
atuacao = 'Criminal'
dados_segmento = dados
dados_segmento = dados_segmento[dados_segmento['tipo_justica'] == nome_justica]
#dados = dados[dados['codtribunal'] == 'TJPE']
dados_segmento = dados_segmento[dados_segmento['Atuacao_Vara'] == atuacao]
print(dados_segmento.shape)
dados_filter = dados_segmento.filter(items=colunas)
#dados_filter['Median(numero_dias)'] = pd.to_numeric(dados_filter['Median(numero_dias)'])
#dados_filter['PIB_percapita'] = pd.to_numeric(dados_filter['PIB_percapita'])
#dados_filter['Count*(npu)'] = pd.to_numeric(dados_filter['Count*(npu)'])
print(dados_filter.dtypes)

(100, 16)
Porte_Municipio                 object
Porte_Tribunal                  object
Classificacao_Acervo_2          object
Atuacao_Vara                    object
Classificacao_PIB_per_capita    object
dtype: object


In [187]:
for x in range(3,9):
    qtd_clusters = x
    #km = KPrototypes(n_clusters=qtd_clusters, init='Huang', n_init=5, verbose=1)
    km = KModes(n_clusters=qtd_clusters, init='Huang', n_init=5, verbose=1)
    clusters = km.fit_predict(dados_filter, categorical=colunas_cat)
    print('Centroídes')
    print(km.cluster_centroids_)
    print('Clusters')
    print(clusters)
    dados_cluster = dados_filter
    #dados_cluster['Cluster'] = clusters
    Nome_cluster = 'jus_' + nome_justica + '_atuacao_' + atuacao + '_cluster_' + str(qtd_clusters)
    dados_segmento[Nome_cluster] = clusters
    print(dados_segmento.head(10))
    numpy.savetxt('Clusters_centroides_' + Nome_cluster + '.csv', km.cluster_centroids_ , fmt = '%s', delimiter=";", header=colunas_str, comments='')
    #Agrupamento
    print("Agrupamento...")
    print(str(x))
    dados_agrupados = dados_segmento.filter(items=[Nome_cluster,'Count*(npu)','Sum(numero_dias)']).groupby(by=[Nome_cluster]).sum()
    
    dados_agrupados['media_duracao'] = dados_agrupados['Sum(numero_dias)'] / dados_agrupados['Count*(npu)']
    print(dados_agrupados)
    atual = 0
    anterior = 0
    valor = 0
    media_quad_dif = 0
    for index, row in dados_agrupados.iterrows():
        anterior = atual
        atual = row['media_duracao']
        if anterior > 0:
            valor += pow(anterior-atual, 2)
    media_quad_dif = math.sqrt(valor/qtd_clusters)
    temp = pd.DataFrame({'Cluster_config':[Nome_cluster],'resultado':[media_quad_dif]})
    resultado = resultado.append(temp)
    
#sns.lmplot(data=dados_cluster.filter(items=colunas_num), x='Median(numero_dias)', y='Count*(npu)', hue='Cluster', 
#                   fit_reg=False, legend=True, legend_out=True)

dados_segmento.to_csv('Clusters_jus_' + nome_justica + '_atuacao_' + atuacao +'_classificacoes.csv', sep = ";")
col = ['codtribunal','cod_orgao_julg','descricao','Cluster_3','Cluster_4','Cluster_5','Cluster_6','Cluster_7','Cluster_8']
dados_somente_orgao_julgador_e_cluster = dados_segmento.filter(items=col)
dados_somente_orgao_julgador_e_cluster.to_csv('Clusters_jus_' + nome_justica + '_atuacao_' + atuacao +'_classificacoes_clusters_somente_orgao_julgador_e cluster.csv', sep = ";")

Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 1, iteration: 1/100, moves: 13, cost: 8.0
Run 1, iteration: 2/100, moves: 1, cost: 8.0
Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 2, iteration: 1/100, moves: 0, cost: 12.0
Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 3, iteration: 1/100, moves: 0, cost: 24.0
Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 4, iteration: 1/100, moves: 2, cost: 16.0
Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 5, iteration: 1/100, moves: 0, cost: 8.0
Best run was number 1
Centroídes
[['Grande_Porte' 'Nao_classificado' 'Acervo Médio' 'Criminal'
  'Municipio com alto PIB per capita']
 ['Grande_Porte' 'Nao_classificado' 'Acervo Grande' 'Criminal'
  'Municipio com médio PIB per capita']
 ['Grande_Porte' 'Nao_classificado' 'Acervo Grande' 'Criminal'
  'Municipio com alto PIB 

Run 2, iteration: 1/100, moves: 5, cost: 1.0
Run 2, iteration: 2/100, moves: 0, cost: 1.0
Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 3, iteration: 1/100, moves: 9, cost: 2.0
Run 3, iteration: 2/100, moves: 0, cost: 2.0
Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 4, iteration: 1/100, moves: 0, cost: 9.0
Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 5, iteration: 1/100, moves: 0, cost: 9.0
Best run was number 2
Centroídes
[['Grande_Porte' 'Nao_classificado' 'Acervo Médio' 'Criminal'
  'Municipio com alto PIB per capita']
 ['Medio_Porte' 'Nao_classificado' 'Acervo Grande' 'Criminal'
  'Municipio com médio PIB per capita']
 ['Grande_Porte' 'Nao_classificado' 'Acervo Grande' 'Criminal'
  'Municipio com médio PIB per capita']
 ['Grande_Porte' 'Nao_classificado' 'Acervo Grande' 'Criminal'
  'Municipio com alto PIB per capita']
 ['Grande_Porte' 'Nao_classificado' 'Acervo Médio

     codtribunal grau  cod_orgao_julg tipo_justica  \
5539        TRF2   G1           11879      Federal   
5540        TRF2   G1           12151      Federal   
5541        TRF2   G1           12239      Federal   
5549        TRF2   G1           12249      Federal   
5552        TRF2   G1           12253      Federal   
5555        TRF2   G1           12257      Federal   
5557        TRF2   G1           12259      Federal   
5558        TRF2   G1           12260      Federal   
5559        TRF2   G1           12261      Federal   
5596        TRF2   G1           12297      Federal   

                                      descricao  uf Porte_Municipio  \
5539  ª VARA FEDERAL CRIMINAL DO RIO DE JANEIRO  RJ    Grande_Porte   
5540  ª VARA FEDERAL CRIMINAL DO RIO DE JANEIRO  RJ    Grande_Porte   
5541  ª VARA FEDERAL CRIMINAL DO RIO DE JANEIRO  RJ    Grande_Porte   
5549  ª VARA FEDERAL CRIMINAL DO RIO DE JANEIRO  RJ    Grande_Porte   
5552  ª VARA FEDERAL CRIMINAL DO RIO DE JANEIRO  R

5                                         1231.793651  
Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 1, iteration: 1/0, moves: 0, cost: 0.0
Centroídes
[['Grande_Porte' 'Nao_classificado' 'Acervo Grande' 'Criminal'
  'Municipio com alto PIB per capita']
 ['Grande_Porte' 'Nao_classificado' 'Acervo Médio' 'Criminal'
  'Municipio com médio PIB per capita']
 ['Medio_Porte' 'Nao_classificado' 'Acervo Grande' 'Criminal'
  'Municipio com alto PIB per capita']
 ['Grande_Porte' 'Nao_classificado' 'Acervo Grande' 'Criminal'
  'Municipio com médio PIB per capita']
 ['Grande_Porte' 'Nao_classificado' 'Acervo Médio' 'Criminal'
  'Municipio com alto PIB per capita']
 ['Medio_Porte' 'Nao_classificado' 'Acervo Grande' 'Criminal'
  'Municipio com médio PIB per capita']]
Clusters
[4 0 0 0 0 0 0 0 0 4 1 1 0 0 4 3 3 0 0 0 0 4 0 0 0 4 0 0 0 4 0 0 0 0 5 5 2
 0 5 0 0 0 0 0 0 0 0 0 0 0 0 0 3 3 3 3 0 0 4 0 0 0 0 0 0 0 0 4 0 0 0 0 0 0
 5 0 0 0 0 4 0 0 0 0 0 0 0 0 3 3 0 0 0 

In [188]:
nome_justica = 'Federal'
atuacao = 'Geral'
dados_segmento = dados
dados_segmento = dados_segmento[dados_segmento['tipo_justica'] == nome_justica]
#dados = dados[dados['codtribunal'] == 'TJPE']
dados_segmento = dados_segmento[dados_segmento['Atuacao_Vara'] == atuacao]
print(dados_segmento.shape)
dados_filter = dados_segmento.filter(items=colunas)
#dados_filter['Median(numero_dias)'] = pd.to_numeric(dados_filter['Median(numero_dias)'])
#dados_filter['PIB_percapita'] = pd.to_numeric(dados_filter['PIB_percapita'])
#dados_filter['Count*(npu)'] = pd.to_numeric(dados_filter['Count*(npu)'])
print(dados_filter.dtypes)

(580, 16)
Porte_Municipio                 object
Porte_Tribunal                  object
Classificacao_Acervo_2          object
Atuacao_Vara                    object
Classificacao_PIB_per_capita    object
dtype: object


In [189]:
for x in range(3,9):
    qtd_clusters = x
    #km = KPrototypes(n_clusters=qtd_clusters, init='Huang', n_init=5, verbose=1)
    km = KModes(n_clusters=qtd_clusters, init='Huang', n_init=5, verbose=1)
    clusters = km.fit_predict(dados_filter, categorical=colunas_cat)
    print('Centroídes')
    print(km.cluster_centroids_)
    print('Clusters')
    print(clusters)
    dados_cluster = dados_filter
    #dados_cluster['Cluster'] = clusters
    Nome_cluster = 'jus_' + nome_justica + '_atuacao_' + atuacao + '_cluster_' + str(qtd_clusters)
    dados_segmento[Nome_cluster] = clusters
    print(dados_segmento.head(10))
    numpy.savetxt('Clusters_centroides_' + Nome_cluster + '.csv', km.cluster_centroids_ , fmt = '%s', delimiter=";", header=colunas_str, comments='')
    #Agrupamento
    print("Agrupamento...")
    print(str(x))
    dados_agrupados = dados_segmento.filter(items=[Nome_cluster,'Count*(npu)','Sum(numero_dias)']).groupby(by=[Nome_cluster]).sum()
    
    dados_agrupados['media_duracao'] = dados_agrupados['Sum(numero_dias)'] / dados_agrupados['Count*(npu)']
    print(dados_agrupados)
    atual = 0
    anterior = 0
    valor = 0
    media_quad_dif = 0
    for index, row in dados_agrupados.iterrows():
        anterior = atual
        atual = row['media_duracao']
        if anterior > 0:
            valor += pow(anterior-atual, 2)
    media_quad_dif = math.sqrt(valor/qtd_clusters)
    temp = pd.DataFrame({'Cluster_config':[Nome_cluster],'resultado':[media_quad_dif]})
    resultado = resultado.append(temp)
    
#sns.lmplot(data=dados_cluster.filter(items=colunas_num), x='Median(numero_dias)', y='Count*(npu)', hue='Cluster', 
#                   fit_reg=False, legend=True, legend_out=True)

dados_segmento.to_csv('Clusters_jus_' + nome_justica + '_atuacao_' + atuacao +'_classificacoes.csv', sep = ";")
col = ['codtribunal','cod_orgao_julg','descricao','Cluster_3','Cluster_4','Cluster_5','Cluster_6','Cluster_7','Cluster_8']
dados_somente_orgao_julgador_e_cluster = dados_segmento.filter(items=col)
dados_somente_orgao_julgador_e_cluster.to_csv('Clusters_jus_' + nome_justica + '_atuacao_' + atuacao +'_classificacoes_clusters_somente_orgao_julgador_e cluster.csv', sep = ";")

Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 1, iteration: 1/100, moves: 74, cost: 154.0
Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 2, iteration: 1/100, moves: 0, cost: 255.0
Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 3, iteration: 1/100, moves: 121, cost: 140.0
Run 3, iteration: 2/100, moves: 32, cost: 140.0
Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 4, iteration: 1/100, moves: 42, cost: 232.0
Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 5, iteration: 1/100, moves: 98, cost: 140.0
Run 5, iteration: 2/100, moves: 0, cost: 140.0
Best run was number 3
Centroídes
[['Grande_Porte' 'Nao_classificado' 'Acervo Grande' 'Geral'
  'Municipio com médio PIB per capita']
 ['Grande_Porte' 'Nao_classificado' 'Acervo Médio' 'Geral'
  'Municipio com alto PIB per capita']
 ['Grande_Porte' 'Nao_classificado

Starting iterations...
Run 1, iteration: 1/100, moves: 42, cost: 55.0
Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 2, iteration: 1/100, moves: 35, cost: 80.0
Run 2, iteration: 2/100, moves: 0, cost: 80.0
Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 3, iteration: 1/100, moves: 13, cost: 55.0
Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 4, iteration: 1/100, moves: 74, cost: 63.0
Run 4, iteration: 2/100, moves: 0, cost: 63.0
Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 5, iteration: 1/100, moves: 12, cost: 55.0
Best run was number 1
Centroídes
[['Medio_Porte' 'Nao_classificado' 'Acervo Grande' 'Geral'
  'Municipio com médio PIB per capita']
 ['Grande_Porte' 'Nao_classificado' 'Acervo Grande' 'Geral'
  'Municipio com médio PIB per capita']
 ['Grande_Porte' 'Nao_classificado' 'Acervo Médio' 'Geral'
  'Municipio com alto PIB per capita']


Run 2, iteration: 1/100, moves: 0, cost: 116.0
Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 3, iteration: 1/100, moves: 1, cost: 42.0
Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 4, iteration: 1/100, moves: 12, cost: 26.0
Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 5, iteration: 1/100, moves: 74, cost: 26.0
Run 5, iteration: 2/100, moves: 0, cost: 26.0
Best run was number 4
Centroídes
[['Grande_Porte' 'Nao_classificado' 'Acervo Médio' 'Geral'
  'Municipio com alto PIB per capita']
 ['Medio_Porte' 'Nao_classificado' 'Acervo Grande' 'Geral'
  'Municipio com médio PIB per capita']
 ['Medio_Porte' 'Nao_classificado' 'Acervo Grande' 'Geral'
  'Municipio com alto PIB per capita']
 ['Grande_Porte' 'Nao_classificado' 'Acervo Grande' 'Geral'
  'Municipio com alto PIB per capita']
 ['Medio_Porte' 'Nao_classificado' 'Acervo Médio' 'Geral'
  'Municipio com alto PIB per capita']
 ['

Init: initializing clusters
Starting iterations...
Run 1, iteration: 1/100, moves: 62, cost: 47.0
Run 1, iteration: 2/100, moves: 12, cost: 47.0
Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 2, iteration: 1/100, moves: 35, cost: 30.0
Run 2, iteration: 2/100, moves: 0, cost: 30.0
Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 3, iteration: 1/100, moves: 0, cost: 115.0
Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 4, iteration: 1/100, moves: 0, cost: 41.0
Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 5, iteration: 1/100, moves: 0, cost: 74.0
Best run was number 2
Centroídes
[['Grande_Porte' 'Nao_classificado' 'Acervo Médio' 'Geral'
  'Municipio com alto PIB per capita']
 ['Medio_Porte' 'Nao_classificado' 'Acervo Grande' 'Geral'
  'Municipio com médio PIB per capita']
 ['Grande_Porte' 'Nao_classificado' 'Acervo Grande' 'Geral'
  'Municipio 

Init: initializing clusters
Starting iterations...
Run 1, iteration: 1/100, moves: 62, cost: 13.0
Run 1, iteration: 2/100, moves: 0, cost: 13.0
Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 2, iteration: 1/100, moves: 0, cost: 62.0
Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 3, iteration: 1/100, moves: 0, cost: 37.0
Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 4, iteration: 1/100, moves: 0, cost: 29.0
Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 5, iteration: 1/100, moves: 103, cost: 13.0
Run 5, iteration: 2/100, moves: 0, cost: 13.0
Best run was number 1
Centroídes
[['Grande_Porte' 'Nao_classificado' 'Acervo Grande' 'Geral'
  'Municipio com médio PIB per capita']
 ['Grande_Porte' 'Nao_classificado' 'Acervo Grande' 'Geral'
  'Municipio com alto PIB per capita']
 ['Medio_Porte' 'Nao_classificado' 'Acervo Médio' 'Geral'
  'Municipio c

In [190]:
#dados['Cluster'] = pd.Series(clusters, index=dados.index)
#dados.head(10)
#colunas_vara_cluster = ['cod_orgao_julg','Cluster']
#dados_join = dados.filter(items=colunas_vara_cluster)
#dados_join.head(10)

In [191]:
#dados_fato = pd.read_csv('log_evento_TJPE_CIVEL_v2.csv',sep=';')

In [192]:
#dados_fato.head(10)

In [193]:
#tab_full = pd.merge(dados_fato, dados_join, how='inner', left_on='oj_cod', right_on='cod_orgao_julg')

In [194]:
#tab_full.head(10)

In [195]:
#dados.head(400)

In [196]:
print(resultado.head(1000))

                            Cluster_config   resultado
0     jus_Estadual_atuacao_Civel_cluster_3  171.618240
0     jus_Estadual_atuacao_Civel_cluster_4  401.362160
0     jus_Estadual_atuacao_Civel_cluster_5  364.200092
0     jus_Estadual_atuacao_Civel_cluster_6  243.228612
0     jus_Estadual_atuacao_Civel_cluster_7  410.053856
0     jus_Estadual_atuacao_Civel_cluster_8  432.371574
0  jus_Estadual_atuacao_Criminal_cluster_3   75.671894
0  jus_Estadual_atuacao_Criminal_cluster_4  184.148653
0  jus_Estadual_atuacao_Criminal_cluster_5  270.004691
0  jus_Estadual_atuacao_Criminal_cluster_6  209.681323
0  jus_Estadual_atuacao_Criminal_cluster_7  354.630364
0  jus_Estadual_atuacao_Criminal_cluster_8  335.107371
0     jus_Estadual_atuacao_Geral_cluster_3  133.742354
0     jus_Estadual_atuacao_Geral_cluster_4  114.826058
0     jus_Estadual_atuacao_Geral_cluster_5  106.033864
0     jus_Estadual_atuacao_Geral_cluster_6  202.065848
0     jus_Estadual_atuacao_Geral_cluster_7  295.529203
0     jus_